Copyright (c) Meta Platforms, Inc. and affiliates.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import os
import sys
sys.path.append("..")
import pickle
import copy
import numpy as np
from nasrec.utils.io_utils import (
    load_pickle_data,
    dump_json
)

In [2]:
ea_results_file = "../www-test/criteo_1shot/criteo-supernet-default-binomial-0.5-xlarge-ea-240gen-128pop-64sample-8childs-default-ft_lr0.04/results.pickle"
ea_results = load_pickle_data(ea_results_file)

In [3]:
!pwd
print(ea_results[-3])

/home/tz86/github/wsnas4rec/notebooks
{'choice': {'micro': [{'active_nodes': [2, 4], 'dense_in_dims': 768, 'sparse_in_dims': 48, 'dense_sparse_interact': 1, 'deep_fm': 1}, {'active_nodes': [1, 5], 'dense_in_dims': 1024, 'sparse_in_dims': 64, 'dense_sparse_interact': 1, 'deep_fm': 1}, {'active_nodes': [2, 5], 'dense_in_dims': 512, 'sparse_in_dims': 64, 'dense_sparse_interact': 0, 'deep_fm': 0}, {'active_nodes': [2, 5], 'dense_in_dims': 16, 'sparse_in_dims': 16, 'dense_sparse_interact': 0, 'deep_fm': 1}, {'active_nodes': [0, 5], 'dense_in_dims': 32, 'sparse_in_dims': 16, 'dense_sparse_interact': 0, 'deep_fm': 1}, {'active_nodes': [2, 4], 'dense_in_dims': 128, 'sparse_in_dims': 32, 'dense_sparse_interact': 1, 'deep_fm': 1}, {'active_nodes': [0, 4], 'dense_in_dims': 1024, 'sparse_in_dims': 64, 'dense_sparse_interact': 0, 'deep_fm': 1}], 'macro': [{'dense_idx': [0], 'sparse_idx': [0], 'dense_left_idx': [0], 'dense_right_idx': [0]}, {'dense_idx': [1], 'sparse_idx': [1, 0], 'dense_left_idx': 

In [4]:
def _convert_to_int32_(array):
    return np.asarray(array).astype(np.int32).tolist()

def _convert_to_float32_(array):
    return np.asarray(array).astype(np.float32).tolist()

def dump_network_config_to_json(item, json_name, 
                                num_blocks: int = 7, 
                                use_layernorm: int = 1,
                                config: str = "xlarge"):
    new_item = copy.deepcopy(item['choice'])
    for idx in range(len(new_item['micro'])):
        new_item['micro'][idx]['active_nodes'] = _convert_to_int32_(
            new_item['micro'][idx]['active_nodes'])
        new_item['micro'][idx]['dense_in_dims'] = _convert_to_int32_(
            new_item['micro'][idx]['dense_in_dims'])
        new_item['micro'][idx]['sparse_in_dims'] = _convert_to_int32_(
            new_item['micro'][idx]['sparse_in_dims'])
        new_item['micro'][idx]['dense_sparse_interact'] = _convert_to_int32_(new_item['micro'][idx]['dense_sparse_interact'])
        new_item['micro'][idx]['deep_fm'] = _convert_to_int32_(new_item['micro'][idx]['deep_fm'])
    new_item['num_blocks'] = num_blocks
    new_item['use_layernorm'] = use_layernorm
    new_item['config'] = config
    new_item['test_loss'] = float(item['test_loss'][0])
    dump_json(json_name, new_item)
    
def sort_ea_results_with_test_loss(ea_results):
    test_losses = [item['test_loss'][0] for item in ea_results]
    sorted_args = np.argsort(test_losses)
    return np.asarray(ea_results)[sorted_args].tolist()

In [ ]:
export_dir = "../ea-criteo-kaggle-xlarge-best-1shot"
# export_dir = "/root/github/wsnas4rec/ea-autoctr-best"
os.makedirs(export_dir, exist_ok=True)

sorted_ea_results = sort_ea_results_with_test_loss(ea_results)
cnt = 0
test_losses = []
last_test_loss = 0.
for i in range(len(sorted_ea_results)):
    cur_test_loss = sorted_ea_results[i]['test_loss'][0]
    if np.abs(cur_test_loss - last_test_loss) > 1e-6:
        last_test_loss = cur_test_loss
        json_path = os.path.join(export_dir, "best_config_{}.json".format(cnt))
        dump_network_config_to_json(sorted_ea_results[i], json_path, config="xlarge")
        cnt += 1
    else:
        pass
    if cnt == 50:
        break

In [5]:
autoctr_config = {"micro": [
    {"active_nodes": [4, 6], "dense_in_dims": 16, "sparse_in_dims": 32, "dense_sparse_interact": 0, "deep_fm": 0},     # Useless. Produces zeros-dense. #S1, N0
    {"active_nodes": [1, 6], "dense_in_dims": 378, "sparse_in_dims": 16, "dense_sparse_interact": 0, "deep_fm": 0},    # Output dense.  S2, N1
    {"active_nodes": [4, 7], "dense_in_dims": 16, "sparse_in_dims": 32, "dense_sparse_interact": 0, "deep_fm": 0},    # Output dense.  S3, N2 (512=32 * 16 (emb))
    {"active_nodes": [1, 6], "dense_in_dims": 378, "sparse_in_dims": 16, "dense_sparse_interact": 0, "deep_fm": 0},    # Output dense.  S4, N3
    {"active_nodes": [1, 6], "dense_in_dims": 378, "sparse_in_dims": 16, "dense_sparse_interact": 0, "deep_fm": 0},    # Output dense  S5, N4
    {"active_nodes": [0, 6], "dense_in_dims": 128, "sparse_in_dims": 16, "dense_sparse_interact": 0, "deep_fm": 0},    # Output dense.  S6, N5
    {"active_nodes": [0, 6], "dense_in_dims": 1024, "sparse_in_dims": 16, "dense_sparse_interact": 0, "deep_fm": 0},   # Output dense  S7, N6
    {"active_nodes": [0, 7], "dense_in_dims": 256, "sparse_in_dims": 256, "dense_sparse_interact": 0, "deep_fm": 0}],  # Output dense. S8, N7
 "macro": [
   {"dense_idx": [0], "sparse_idx": [0], "dense_left_idx": [0], "dense_right_idx": [0]},
   {"dense_idx": [1], "sparse_idx": [0], "dense_left_idx": [0], "dense_right_idx": [0]},
   {"dense_idx": [0], "sparse_idx": [0], "dense_left_idx": [1], "dense_right_idx": [1]},
   {"dense_idx": [0, 2], "sparse_idx": [0], "dense_left_idx": [2], "dense_right_idx": [2]},
   {"dense_idx": [1], "sparse_idx": [0, 3], "dense_left_idx": [3], "dense_right_idx": [3]},
   {"dense_idx": [4, 5], "sparse_idx": [0], "dense_left_idx": [4], "dense_right_idx": [4]},
   {"dense_idx": [0, 2, 4, 5, 6], "sparse_idx": [0], "dense_left_idx": [5], "dense_right_idx": [5]},
   {"dense_idx": [0, 2, 4, 6, 7], "sparse_idx": [0], "dense_left_idx": [6], "dense_right_idx": [6]}],
   "num_blocks": 8, "config": "xlarge-zeros", "use_layernorm": 1}

dump_json("../nasrec/configs/autoctr_config.json", autoctr_config)

In [ ]:
!pwd